In [2]:
import scipy.io
import numpy as np

# Path to your .mat file
mat_file_path = 'save_data_AVG.mat'
#mat_file_path = '../dannce/demo/markerless_mouse_1/COM/predict_results/com3d.mat'

# Load the .mat file
mat_contents = scipy.io.loadmat(mat_file_path)

# Display the contents
for key, value in mat_contents.items():
    #if not key.startswith('__'):
    print(f"Key: {key}")
    print(len(value))


Key: __header__
73
Key: __version__
3
Key: __globals__
0
Key: pred
360000
Key: data
360000
Key: p_max
360000
Key: sampleID
1
Key: metadata
1


In [4]:
frames = mat_contents["pred"][1:1000]
data = {}
data["pred"] = frames

scipy.io.savemat("test_pred_only.mat", data)

In [14]:
print(len(mat_contents["metadata"][0][0]))
for d in mat_contents["metadata"][0][0]:
    print(d)

94
[[0]]
[[0]]
[[(array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_21_1/20201221_163226_Label3D_dannce.mat'],
        dtype='<U115'),)                                                                                                         ]]
['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_22_1/COM/predict00/com3d.mat']
['./COM/predict00']
['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_21_1/COM/train00/fullmodel_weights/fullmodel_end.hdf5']
['./COM/train00']
['./DANNCE/predict03/']
['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/bud/2021_06_21_1/DANNCE/train02/weights.178-2.65476.hdf5']
['./DANNCE/train02/']
[[array([[(array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/test/2/COM/predict00/com3d.mat'],
                dtype='<U81'), array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/test/2/20201107_164529_Label3D_dannce.mat'],
                dtype='<U92'))  

In [ ]:
print(len(mat_contents["sampleID"][0]))
print(mat_contents["sampleID"])

In [17]:
print((mat_contents["pred"][0]))

[[415.78637695 364.45196533 389.87600708 361.69506836 343.89483643
  324.03741455 315.55377197 347.51333618 345.45239258 347.87133789
  349.9324646  378.03158569 385.50210571 384.97793579 387.8006897
  316.05712891 323.0378418  321.77130127 330.42373657 346.02996826
  364.62158203 346.48086548 366.78930664]
 [114.58620453  97.39944458  80.85489655  56.98851776  12.65446758
  -16.89956474 -23.19591141  66.76320648  70.80370331  79.02404785
   86.8793335   51.16601944  55.50511169  69.89868164  80.01965332
    2.39903808  27.82298279   8.13162136  28.03132439 -16.14893723
   -0.90713215 -12.36277771  -2.2607131 ]
 [ 66.22083282  72.17269897  79.75028229  81.79259491  85.07378387
   57.34063339  22.68009186  65.99371338  38.04782867  20.5947094
    9.1001358   65.11450195  36.41916275  20.37991714   8.82514381
   48.29601669  36.77716446  13.33489132   9.15814972  48.69794464
   33.57286453  12.66648197   8.71748257]]
